* Main motivation of streamlining the process of reading data
* TFRecords - efficient Tensorflow file format
* We will learn how to read & write on TFRecords
* Reading & Writing on queues
* Multithreading

### Input Pipeline
* For larger data, load chunks only when they are needed
* Keep the rest in file system
* Pre-processing of data is independent of training
* TFRedocrds are suppose of handle such situation

In [1]:
import os
import tensorflow as tf

In [2]:
from tensorflow.contrib.learn.python.learn.datasets import mnist

In [3]:
save_dir = "."

In [4]:
data_sets = mnist.read_data_sets(save_dir, dtype= tf.uint8, reshape=False, validation_size=1000)

Extracting .\train-images-idx3-ubyte.gz
Extracting .\train-labels-idx1-ubyte.gz
Extracting .\t10k-images-idx3-ubyte.gz
Extracting .\t10k-labels-idx1-ubyte.gz


In [15]:
data_sets[0].images[0]

array([[[  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0]],

       [[  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0]],

       [[  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
        [  0],
      

In [17]:
data_splits = ["train","test","validation"]

In [18]:
for d in range(3):
    print ("saving " + data_splits[d])
    data_set = data_sets[d]
    
    filename = os.path.join(save_dir, data_splits[d] + '.tfrecords')
    writer = tf.python_io.TFRecordWriter(filename)
    for index in range(data_set.images.shape[0]):
        image = data_set.images[index].tostring()
        example = tf.train.Example(features = tf.train.Features(feature={
            'height':tf.train.Feature(int64_list = tf.train.Int64List(value=[data_set.images.shape[1]])),
            'width':tf.train.Feature(int64_list = tf.train.Int64List(value=[data_set.images.shape[2]])),
            'depth':tf.train.Feature(int64_list = tf.train.Int64List(value=[data_set.images.shape[3]])),
            'label':tf.train.Feature(int64_list = tf.train.Int64List(value=[data_set.labels[index]])),
            'raw_image':tf.train.Feature(bytes_list = tf.train.BytesList(value=[image]))
            
        }))
        
        writer.write(example.SerializeToString())
writer.close()

saving train
saving test
saving validation


In [12]:
data_sets[0].images.shape

(59000, 28, 28, 1)

In [19]:
reader = tf.TFRecordReader()

In [21]:
filename = os.path.join(save_dir,"train.tfrecords")
filename_queue = tf.train.string_input_producer([filename],num_epochs=10)

In [23]:
_, serialized_data = reader.read(filename_queue)

In [24]:
serialized_data

<tf.Tensor 'ReaderReadV2_1:1' shape=() dtype=string>

In [26]:
features = tf.parse_single_example(serialized_data, features={
    'image_raw': tf.FixedLenFeature([],tf.string),
    'label':tf.FixedLenFeature([],tf.int64)
})

In [27]:
features

{'image_raw': <tf.Tensor 'ParseSingleExample/Squeeze_image_raw:0' shape=() dtype=string>,
 'label': <tf.Tensor 'ParseSingleExample/Squeeze_label:0' shape=() dtype=int64>}

In [14]:
data_sets[0].labels.shape

(59000,)

### Queue Creation

In [28]:
sess = tf.InteractiveSession()
#shapes - telling the size of items in advance
queue1 = tf.FIFOQueue(capacity=10, dtypes=[tf.string],shapes=[()])

In [29]:
enque_op = queue1.enqueue(["Z"])

In [30]:
sess.run(queue1.size())

0

In [31]:
enque_op.run()

In [32]:
sess.run(queue1.size())

1

In [7]:
enque_op = queue1.enqueue(["E"])
enque_op.run()
enque_op = queue1.enqueue(["K"])
enque_op.run()
enque_op = queue1.enqueue(["E"])
enque_op.run()

In [8]:
sess.run(queue1.size())

4

In [9]:
x = queue1.dequeue()

In [10]:
x.eval()

b'Z'

In [11]:
x = queue1.dequeue()
x.eval()

b'E'

In [12]:
x = queue1.dequeue()
x.eval()

b'K'

In [19]:
enque_op = queue1.enqueue(["E"])
enque_op.run()
enque_op = queue1.enqueue(["K"])
enque_op.run()
enque_op = queue1.enqueue(["E"])
enque_op.run()

In [20]:
inputs = queue1.dequeue_many(2)

In [21]:
inputs.eval()

array([b'E', b'K'], dtype=object)

In [22]:
#Tensorflow session is multi threaded
#Multiple threads can use the same session
#Run ops in parallel

In [33]:
import threading

In [34]:
genral_random_normal = tf.random_normal(shape=())

queue = tf.FIFOQueue(capacity=100, dtypes=[tf.float32], shapes=())
enque = queue.enqueue(genral_random_normal)

In [27]:
def add():
    for i in range(10):
        sess.run(enque)

In [30]:
threads = [threading.Thread(target=add, args=()) for i in range(10)]

In [31]:
for t in threads:
    t.start()

In [32]:
sess.run(queue.size())

100

In [34]:
x = queue.dequeue_many(50)
x.eval()

array([-0.91700041,  1.06900203, -1.25437331, -0.54801881, -0.38989627,
       -0.46762475,  0.88529086, -1.05124879, -0.81140858,  1.25654447,
       -0.01337299, -1.50150537,  0.52652627,  0.16372143, -0.4275195 ,
        0.19320062, -0.77131397,  0.45199049, -0.96952766, -0.58470839,
       -1.97644472, -2.25858378, -0.33022475, -0.54432708,  0.88813865,
        0.298397  , -0.48165751, -1.10197115, -0.84567058, -0.50833195,
        0.07419065,  0.34973881, -0.06683222, -0.47225034, -1.27323937,
       -0.88559961, -0.85171402,  1.86602545, -0.11018598,  0.29020962,
        0.81564206, -1.16399121, -1.08930326, -1.00915325,  0.87850976,
       -0.09321502, -0.49639085,  0.47721642,  0.18031943,  1.38161182], dtype=float32)

In [35]:
gen_random_normal = tf.random_normal(shape=())

In [36]:
queue = tf.RandomShuffleQueue(capacity=100, dtypes=[tf.float32], min_after_dequeue=1)

In [37]:
enqueue_op = queue.enqueue(gen_random_normal)

In [38]:
qr = tf.train.QueueRunner(queue,[enqueue_op] * 4)

In [39]:
coord = tf.train.Coordinator()

In [41]:
queue_threads = qr.create_threads(sess, coord=coord, start=True)

In [42]:
coord.request_stop()

In [43]:
coord.join(queue_threads)

In [44]:
reader = tf.TFRecordReader()

In [45]:
filename_q = tf.train.string_input_producer(["train.tfrecords"])

In [47]:
ser  = reader.read(filename_q)

In [50]:
ser = ser[-1]

In [52]:
features = tf.parse_single_example(ser, features={
    'image_raw': tf.FixedLenFeature([], tf.string),
    'label' : tf.FixedLenFeature([], tf.int64)
})

In [ ]:
sess.run(features)

### Coordinator & QueueRunner
* Coordinator for coordinating the termination of a set of threads
* QueueRunner for getting multiple threads enqueue data to same queue seamlessly

In [35]:
gen_random_normal = tf.random_normal(shape=())
queue = tf.FIFOQueue(capacity=100, dtypes=[tf.float32],shapes=())
enque = queue.enqueue(genral_random_normal)

In [36]:
def add(coord,i):
    while not coord.should_stop():
        sess.run(enque)
        if i == 11:
            coord.request_stop()

In [37]:
coord = tf.train.Coordinator()

In [38]:
threads = [threading.Thread(target=add, args=(coord,i)) for i in range(10)]
coord.join(threads)

In [39]:
for t in threads:
    t.start()

In [40]:
sess.run(queue.size())

100

In [62]:
gen_random_normal = tf.random_normal(shape=())
queue = tf.RandomShuffleQueue(capacity=100, dtypes=[tf.float32],min_after_dequeue=1)
enque_op = queue.enqueue(genral_random_normal)
enque_op2 = queue.enqueue(5)

In [63]:
rq = tf.train.QueueRunner(queue, [enque_op, enque_op2])

In [64]:
coord = tf.train.Coordinator()

In [65]:
threads = rq.create_threads(sess,coord=coord, start=True)


In [ ]:
rq.

In [60]:
[enque_op] * 5

[<tf.Operation 'random_shuffle_queue_enqueue' type=QueueEnqueueV2>,
 <tf.Operation 'random_shuffle_queue_enqueue' type=QueueEnqueueV2>,
 <tf.Operation 'random_shuffle_queue_enqueue' type=QueueEnqueueV2>,
 <tf.Operation 'random_shuffle_queue_enqueue' type=QueueEnqueueV2>,
 <tf.Operation 'random_shuffle_queue_enqueue' type=QueueEnqueueV2>]

In [66]:
threads

[<Thread(Thread-38, started 19636)>,
 <Thread(Thread-39, started 6140)>,
 <Thread(Thread-40, started 7696)>]